In [4]:
# LangChain에서 제공하는 ChatOpenAI 모듈을 import
from langchain.chat_models import ChatOpenAI

# ChatOpenAI 클래스의 인스턴스를 생성하여 'chat' 객체에 저장
chat = ChatOpenAI()

# 'predict' 메서드를 사용하여 질문을 던지고, AI의 응답을 받아 'answer' 변수에 저장
answer = chat.predict("How many planets are there?")

answer

'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'